## Point Registration

This tutorial shows you how to register 'nii' files with points. The points can be computed by segmentations like in Verse19.

You can get the Verse19 data from:

https://osf.io/nqjyw/

We assume in this tutorial that you copied the "dataset-verse19validation" folder next to this file.


In [1]:
from pathlib import Path
from TPTBox.registration.ridged_points import ridged_points_from_subreg_vert
from TPTBox import NII, POI, calc_centroids_from_subreg_vert
from TPTBox.snapshot2D import Snapshot_Frame, create_snapshot
from IPython.display import Image
import numpy as np

import SimpleITK
from SimpleITK import VersorRigid3DTransform

loading data

In [10]:
ct1 = Path(
    "/media/data/robert/datasets/dataset-neuropoly/rawdata_upscale/sub-m002126/ses-20100716/anat/sub-m002126_ses-20100716_acq-sag_desc-stitched_T2w.nii.gz"
).absolute()
ct2 = Path(
    "/media/data/robert/datasets/dataset-neuropoly/rawdata/sub-m002126/ses-20100716/anat/sub-m002126_ses-20100716_acq-ax_chunk-2_T2w.nii.gz"
).absolute()


if not ct1.exists():
    ct1 = Path(
        "/DATA/NAS/datasets_processed/MRI_spine/dataset-SpineGAN/dataset-spinegan_T2w_all/rawdata/spinegan0032/T2w/sub-spinegan0032_ses-20210705_sequ-302_part-inphase_dixon.nii.gz"
    )
    ct2 = Path(
        "/DATA/NAS/datasets_processed/MRI_spine/dataset-SpineGAN/dataset-spinegan_T2w_all/rawdata/spinegan0032/CT/sub-spinegan0032_ses-20210623_sequ-12_ct.nii.gz"
    )


def to_vert(p: Path):
    a = str(p).replace("rawdata", "derivatives").replace("_part-inphase", "_part-inphase_mod-dixon")
    return Path(a.rsplit("_", maxsplit=1)[0] + "_seg-vert_msk.nii.gz")


def to_poi(p: Path):
    return Path(str(p).replace("rawdata", "derivatives").replace("_msk.nii.gz", "_ctd.json"))


def to_subreg(p: Path):
    a = Path(str(p).replace("rawdata", "derivatives").replace("_seg-vert_msk.nii.gz", "_seg-spine_msk.nii.gz"))
    if a.exists():
        return a
    return Path(str(p).replace("rawdata", "derivatives").replace("_seg-vert_msk.nii.gz", "_seg-subreg_msk.nii.gz"))


c1_vert = to_vert(ct1)

c2_vert = to_vert(ct2)

c1_subreg = to_subreg(c1_vert)
c2_subreg = to_subreg(c2_vert)
print(c1_subreg)
c1_poi = to_poi(c1_subreg)
c2_poi = to_poi(c2_subreg)

print(ct1.exists(), ct1)
print(c1_vert.exists(), c2_vert.exists())
print(c1_subreg.exists(), c2_subreg.exists())
print(c1_poi.exists(), c2_poi.exists())
print(c1_poi)
# /DATA/NAS/datasets_processed/MRI_spine/dataset-SpineGAN/dataset-spinegan_T2w_all/derivatives/spinegan0032/T2w/sub-spinegan0032_ses-20210623_sequ-12_seg-vert_msk.nii.gz
# /DATA/NAS/datasets_processed/MRI_spine/dataset-SpineGAN/dataset-spinegan_T2w_all/derivatives/spinegan0032/T2w/sub-spinegan0032_ses-20210623_sequ-12_seg-vert_msk.nii.gz

/DATA/NAS/datasets_processed/MRI_spine/dataset-SpineGAN/dataset-spinegan_T2w_all/derivatives/spinegan0032/T2w/sub-spinegan0032_ses-20210705_sequ-302_part-inphase_mod-dixon_seg-subreg_msk.nii.gz
True /DATA/NAS/datasets_processed/MRI_spine/dataset-SpineGAN/dataset-spinegan_T2w_all/rawdata/spinegan0032/T2w/sub-spinegan0032_ses-20210705_sequ-302_part-inphase_dixon.nii.gz
True True
True True
True True
/DATA/NAS/datasets_processed/MRI_spine/dataset-SpineGAN/dataset-spinegan_T2w_all/derivatives/spinegan0032/T2w/sub-spinegan0032_ses-20210705_sequ-302_part-inphase_mod-dixon_seg-subreg_ctd.json


### From Centroid or Points of Interest(POI) list

We load the POIs form the json. It should have the following shape:
```[
    {
        "direction": ["P","I","R"]
    },
    {
        "label": 20, "X": 91.1,"Y": 40.0, "Z": 95.0
    },
    {
        "label": 21, "X": 76.5,"Y": 68.1, "Z": 90.3
    },
    {
        "label": 22, "X": 63.6,"Y": 101.2, "Z": 86.3
    },
    {
        "label": 23, "X": 59.0,"Y": 136.8, "Z": 86.8
    },
    {
        "label": 24, "X": 72.1,"Y": 166.6, "Z": 94.0
    }
]
```

where the coordinate are the local cords of the image

In [21]:
from TPTBox.core.nii_wrapper import to_nii
from TPTBox import Location

ct1_nii_org = to_nii(ct1, False)  # .resample_from_to(c1_vert)
ct2_nii = to_nii(ct2, False)  # .resample_from_to(c2_vert)
c1_vert_nii_org = to_nii(c1_vert, True)

In [25]:
from TPTBox.core.nii_wrapper import to_nii

orientation = ct2_nii.orientation  # ("L", "A", "S")
zoom = ct2_nii.zoom  # (0.8571, 0.8571, 0.8571)
c2_poi_ = calc_centroids_from_subreg_vert(c2_vert, c2_subreg, subreg_id=[42, 50])
poi = c2_poi_.copy()

resample_filter = ridged_points_from_subreg_vert(
    c2_poi_,
    c1_vert,
    c1_subreg,
    c1_poi,
    orientation=orientation,
    zoom=zoom,
    subreg_id=[50],
    c_val=0,
    verbose=False,
    save_buffer_file=False,
)
rep_moving_nii: NII = resample_filter.transform_nii(to_nii(ct2, False))
rep_seg_moving_nii: NII = resample_filter.transform_nii(to_nii(c2_vert, True))
rep_seg2_moving_nii: NII = resample_filter.transform_nii(to_nii(c2_subreg, True))
print(poi[22, 50])
poi_new = resample_filter.transform_poi(c2_poi_)
print(poi_new[22, 50])
c1_vert_iso = to_nii(c1_vert, True).rescale(zoom)
p1 = POI.load(c1_poi).rescale(zoom).extract_subregion_(50, 42)
c1_iso = to_nii(ct1, False).rescale(zoom)
# calc crop
# crop = rep_moving_nii.compute_crop()
# crop = c1_iso.compute_crop(other_crop=crop)

ct1_frame = Snapshot_Frame(c1_iso, segmentation=rep_seg_moving_nii.resample_from_to(c1_iso), centroids=p1)
ct2_frame = Snapshot_Frame(rep_moving_nii, segmentation=c1_vert_iso.resample_from_to(rep_moving_nii), centroids=p1)
ct3_frame = Snapshot_Frame(
    to_nii(ct2, False).resample_from_to(rep_seg_moving_nii), segmentation=rep_seg_moving_nii, centroids=poi_new.copy()
)
ct_org_frame = Snapshot_Frame(
    to_nii(ct2, False).resample_from_to(c1_iso),
    segmentation=c1_vert_iso,
    centroids=poi_new,  # c2_poi_.copy().resample_from_to(c1_iso)
)
create_snapshot("test.jpg", [ct1_frame, ct2_frame, ct3_frame, ct_org_frame])
Image(filename="test.jpg")

to_nii(ct1, False).save("/DATA/NAS/ongoing_projects/robert/test/fixed.nii.gz")
to_nii(c1_vert, False).save("/DATA/NAS/ongoing_projects/robert/test/fixed_vert.nii.gz")
rep_moving_nii.save("/DATA/NAS/ongoing_projects/robert/test/rep_moving_nii.nii.gz")
rep_seg_moving_nii.save("/DATA/NAS/ongoing_projects/robert/test/rep_moving_vert_nii.nii.gz")
rep_seg2_moving_nii.save("/DATA/NAS/ongoing_projects/robert/test/rep_moving_subreg_nii.nii.gz")

[ ] Calc centroids from subregion id [Spinosus_Process, Vertebra_Corpus] (512, 1005, 122)
list
[ ] Calc centroids from subregion id Location.Spinosus_Process (512, 1005, 122)
[ ] Calc centroids from subregion id Location.Vertebra_Corpus (512, 1005, 122)
load
[ ] Calc centroids from subregion id [Vertebra_Corpus] (960, 960, 15)
list
[ ] Calc centroids from subregion id Location.Vertebra_Corpus (960, 960, 15)
(291.11, 363.01, 71.39)
(310.933824569798, 425.68625885696036, 14.315093413085252)
[ ] Rescaled centroid coordinates to spacing (x, y, z) = (0.5234375, 0.5233831, 2.0003345) mm
[ ] resample_from_to: shp=(688, 688, 25); ori=('P', 'S', 'R'), zoom=(0.52, 0.52, 2.0), seg=True to shp=(688, 688, 25); ori=('P', 'S', 'R'), zoom=(0.52, 0.52, 2.0), seg=False
[ ] resample_from_to: shp=(688, 688, 25); ori=('P', 'S', 'R'), zoom=(0.52, 0.52, 2.0), seg=True to shp=(688, 688, 25); ori=('P', 'S', 'R'), zoom=(0.52, 0.52, 2.0), seg=False
[ ] resample_from_to: shp=(512, 1005, 122); ori=('P', 'S', 'R'),

In [20]:
from TPTBox.core.sitk_utils import get_ras_affine_from_sitk_meta, get_sitk_metadata_from_ras_affine

aff = resample_filter.get_affine()


def get_translation(self):
    # T(x) = A ( x - c ) + (t + c)
    # T(x) = Ax (- Ac + t + c)
    # with the first 9 parameters filling A in row-major order, the last 3 filling t, and the fixed parameters filling c.
    v = self._transform
    A = np.eye(4)  # noqa: N806
    A[:3, :3] = -np.array(v.GetInverse().GetMatrix()).reshape(3, 3)
    c = np.array(v.GetInverse().GetCenter())
    t = np.array(v.GetInverse().GetTranslation())
    trans = A[:3, :3] @ c + c + t
    A[:3, 3] = trans
    return trans, A, c, t


origin, spacing, direction = get_sitk_metadata_from_ras_affine(ct2_nii.affine)
trans, A, c, t = get_translation(resample_filter)
origin = tuple(np.array(origin) + trans)
direction = A[:3, :3].T @ (np.array(direction).reshape(3, 3))
affine = get_ras_affine_from_sitk_meta(spacing, direction.reshape(-1), origin)
# affine = ct2_nii.affine @ aff
out = ct2_nii.copy()
out._unpack()
out.affine = affine
out.save("/DATA/NAS/ongoing_projects/robert/test.nii.gz")

[*] Save /DATA/NAS/ongoing_projects/robert/test.nii.gz as int16


In [35]:
from TPTBox.registration.ridged_points.point_registration import Point_Registration

np.set_printoptions(precision=2, floatmode="fixed", suppress=True)


def get_affine(self: Point_Registration, poi):
    # T(x) = A ( x - c ) + (t + c)
    # T(x) = Ax (- Ac + t + c)
    # with the first 9 parameters filling A in row-major order, the last 3 filling t, and the fixed parameters filling c.
    v = self._transform
    # v.SetTranslation([0, 0, 0])
    # v.SetFixedParameters([0, 0, 0])
    # v.SetMatrix([1, 0, 0, 0, 1, 0, 0, 0, 1])
    print(v)
    x_trans = self.transform_poi(poi)
    print(np.array(x_trans[22, 50]))
    x, y, z = poi[22, 50]
    ctr_b = self._img_moving.TransformContinuousIndexToPhysicalPoint((x, y, z))
    ctr_b = self._transform.GetInverse().TransformPoint(ctr_b)
    ctr_b = self._img_fixed.TransformPhysicalPointToContinuousIndex(ctr_b)
    print(ctr_b)
    # A = np.eye(4)  # noqa: N806
    # A[:3, :3] = np.array(v.GetInverse().GetMatrix()).reshape(3, 3)
    # c = np.array(v.GetInverse().GetCenter())
    # t = np.array(v.GetInverse().GetTranslation())
    # trans = -A[:3, :3] @ c + c + t
    # A[:3, 3] = trans
    # x, y, z = poi[22, 50]
    A = resample_filter.get_affine()
    ctr_b = self._img_moving.TransformContinuousIndexToPhysicalPoint((x, y, z))
    x = np.ones((4,))
    x[:3] = np.array(ctr_b)
    ctr_b = (A @ x)[:3].tolist()  # self._transform.GetInverse().TransformPoint(ctr_b)
    ctr_b = self._img_fixed.TransformPhysicalPointToContinuousIndex(ctr_b)
    print(ctr_b)
    # v: SimpleITK.VersorRigid3DTransform = self._transform
    ## v = v.GetInverse()
    ## print(v.GetInverse())
    # A[3, 3] = 1

    # A[:3, 3] = trans
    # x = resample_filter._img_moving.TransformContinuousIndexToPhysicalPoint(x)
    # x = v.GetInverse().TransformPoint(x)
    # x = resample_filter._img_fixed.TransformContinuousIndexToPhysicalPoint(x)

    # return A


# ctr_b = self._img_moving.TransformContinuousIndexToPhysicalPoint((x, y, z))
# ctr_b = self._transform.GetInverse().TransformPoint(ctr_b)
# ctr_b = self._img_fixed.TransformPhysicalPointToContinuousIndex(ctr_b)
# out[key, key2] = ctr_b
# x = np.ones((4,))
# print(x)
#
#

get_affine(resample_filter, c2_poi_)

# (289.36, 344.41, 22.32)
# (20.206176606679104, 187.6635049680779, 489.73278324492975)
# print(get_affine(resample_filter))
# print(resample_filter._transform)
# centroids={}, orientation=('L', 'A', 'S'), zoom=(0.21, 0.21, 5.0), shape=(576, 576, 25), info={}, origin=(53.9278374, -42.8728638, -80.66082)

itk::simple::VersorRigid3DTransform
 VersorRigid3DTransform (0x1e4c640)
   RTTI typeinfo:   itk::VersorRigid3DTransform<double>
   Reference Count: 3
   Modified Time: 4225
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     0.999384 -0.0123232 -0.0328746 
     0.0134796 0.99929 0.0351875 
     0.0324177 -0.035609 0.99884 
   Offset: [-24.5722, -223.837, 15.6594]
   Center: [-8.28761, 57.6167, -348.094]
   Translation: [-13.8337, -236.239, 13.7429]
   Inverse: 
     0.999384 0.0134796 0.0324177 
     -0.0123232 0.99929 -0.035609 
     -0.0328746 0.0351875 0.99884 
   Singular: 0
   Versor: [ -0.0177046, -0.0163281, 0.00645271, 0.999689 ]

[162.75 222.80  28.64]
(162.75460167334307, 222.79690446232468, 28.63527893306818)
(162.75460167334307, 222.7969044623248, 28.63527893306818)


# Cropping
The moved images has some space unoccupied. We can remove non corresponding spaces by cropping

In [ ]:
### OUTDATED ###

from BIDS import to_nii, NII

# orientation = rep_moving_nii.orientation
# zoom = rep_moving_nii.zoom


# NII is a wrapper around nibabel
ct1_nii = to_nii(ct1).reorient_(orientation).rescale_(zoom)
ct2_nii = rep_moving_nii.copy()
ct1_seg = to_nii(c1_vert, seg=True).reorient_(orientation).rescale_(zoom)
ct2_seg = rep_seg_moving_nii.copy()
assert ct1_nii.shape == ct2_nii.shape, (ct1_nii, ct2_nii)
ex_slice_f = ct1_nii.compute_crop()
shared_pixels = ct2_nii.compute_crop(other_crop=ex_slice_f)


c1_poi_cropped = POI.load(c1_poi).reorient_(orientation).rescale_(zoom).apply_crop(shared_pixels)
ct1_nii.apply_crop_(shared_pixels)
ct2_nii.apply_crop_(shared_pixels)
ct1_seg.apply_crop_(shared_pixels)
ct2_seg.apply_crop_(shared_pixels)

ct1_frame = Snapshot_Frame(ct1_nii, segmentation=ct1_seg, centroids=c1_poi_cropped)
ct2_frame = Snapshot_Frame(ct2_nii, segmentation=ct2_seg, centroids=c1_poi_cropped)
create_snapshot("test.jpg", [ct1_frame, ct2_frame])
Image(filename="test.jpg")

## Computing Centroids from segmentation

We can generate Centroids/POI by computing the Center Of Mass of Segmentation.

Variant 1: Just using the segmentation

In [ ]:
from TPTBox import calc_centroids

ctd = calc_centroids(c1_vert, decimals=0)
print(to_nii(c1_vert))
print("orientation", ctd.orientation)
print("shape", ctd.shape)
print("zoom", ctd.zoom)
print("points", ctd.keys())
print("id 9", ctd[9, 50])
print("\n", ctd)

Variant 2: Segmentation and Subregion Segmentation

If you have two files, where the second splits the other in smaller chunks you can use this variant. (Verse does not provide this...)

In [ ]:
instance_nii = to_nii(c1_vert, seg=True)
arr = instance_nii.get_array()
arr[arr != 0] = 50
subregion_nii = instance_nii.set_array(arr, inplace=False)
from TPTBox import calc_centroids_from_subreg_vert

ctd = calc_centroids_from_subreg_vert(instance_nii, subregion_nii, subreg_id=[50])
print(ctd)
print(list(ctd.items()))
print("Note: the ids are subregion_ID*256 + segmentation id. Matching points MUST have the same ID")